***
### Import of libraries and data
***

##### Libraries

In [1]:
from traffic.core import Traffic
import utils.helperfunctions as hf
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import plotly.express as px
import plotly.graph_objects as go

import plotly.graph_objects as go
from traffic.data import navaids, airports

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

##### Data

In [2]:
t = Traffic.from_file(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/landing_all.parquet"
)
# t_26 = t.query("rwy == '26L' or rwy == '26R'")

In [3]:
t_26_l = t.query("rwy == '26L'")
t_26_r = t.query("rwy == '26R'")
t_08_l = t.query("rwy == '08L'")
t_08_r = t.query("rwy == '08R'")

In [4]:
t_26_l.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rwy26_l.parquet")
t_26_r.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rwy26_r.parquet")
t_08_l.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rwy08_l.parquet")
t_08_r.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rwy08_r.parquet")

***
### ROKIL1B
***

#### Reduction to flights passing ROKIL

In [ ]:
t_ROKIL1B = (
    t_26.iterate_lazy()
    .pipe(hf.aligned_navpoint, "ROKIL")
    .eval(desc="aligned_ROKIL", max_workers=20)
)
t_ROKIL1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rokil.parquet"
)

#### Remove go-arounds

In [ ]:
t_ROKIL1B = (
    t_ROKIL1B.iterate_lazy()
    .pipe(hf.remove_ga, "EDDM")
    .eval(desc="removing GAs", max_workers=20)
)
t_ROKIL1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rokil_ga.parquet"
)
t_ROKIL1B.to_parquet(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rokil_ga.parquet")

#### Crop at threshold

In [ ]:
t_ROKIL1B_26L = t_ROKIL1B.query("rwy == '26L'")
t_ROKIL1B_26R = t_ROKIL1B.query("rwy == '26R'")

t_ROKIL1B_26L = (
    t_ROKIL1B_26L.iterate_lazy()
    .pipe(hf.crop_after_th, "EDDM", "26L")
    .eval(desc="Cropping at TH 26L", max_workers=20)
)

t_ROKIL1B_26R = (
    t_ROKIL1B_26R.iterate_lazy()
    .pipe(hf.crop_after_th, "EDDM", "26R")
    .eval(desc="Cropping at TH 26R", max_workers=20)
)

t_ROKIL1B = t_ROKIL1B_26L + t_ROKIL1B_26R

t_ROKIL1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rokil_ga_th.parquet"
)

#### Crop before waypoint

In [ ]:
t_ROKIL1B = (
    t_ROKIL1B.iterate_lazy()
    .pipe(hf.crop_before_wp, "ROKIL")
    .eval(desc="cropped_before_ROKIL", max_workers=20)
)

t_ROKIL1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rokil_ga_th_wp.parquet"
)

#### Map-plot

In [ ]:
# Load the data
t_ROKIL = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rokil_ga_th_wp.parquet")

# Create a single figure for the map
fig = go.Figure()

# Add ROKIL STAR 26R
rokil_lat = [
    navaids["ROKIL"].latitude,
    navaids["DM421"].latitude,
    navaids["DM429"].latitude,
    navaids["DM439"].latitude,
    airports["EDDM"].runways.data.query(f"name == '26R'").latitude.iloc[0],
]
rokil_lon = [
    navaids["ROKIL"].longitude,
    navaids["DM421"].longitude,
    navaids["DM429"].longitude,
    navaids["DM439"].longitude,
    airports["EDDM"].runways.data.query(f"name == '26R'").longitude.iloc[0],
]
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=rokil_lat,
        lon=rokil_lon,
        name="ROKIL1B STAR",
        line=dict(width=10, color="red"),
        opacity=0.4,
        showlegend=True,
    )
)

# Add ROKIL STAR 26L
rokil_lat = [
    navaids["DM439"].latitude,
    navaids["DM449"].latitude,
    airports["EDDM"].runways.data.query(f"name == '26L'").latitude.iloc[0],
]
rokil_lon = [
    navaids["DM439"].longitude,
    navaids["DM449"].longitude,
    airports["EDDM"].runways.data.query(f"name == '26L'").longitude.iloc[0],
]
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=rokil_lat,
        lon=rokil_lon,
        name="ROKIL STAR",
        line=dict(width=10, color="red"),
        opacity=0.4,
        showlegend=False,
    )
)

# Add observed trajectories
i = 0
for flight in t_ROKIL.sample(100):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=1.5, color="#757ef3"),
            opacity=0.5,
            name="Observed trajectories",
            showlegend=False if i > 0 else True,
        )
    )
    i += 1

# Add ROKIL navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["ROKIL"].latitude],
        lon=[navaids["ROKIL"].longitude],
        marker=dict(size=10, color="red"),
        text=["ROKIL"],
        textposition="top left",
        textfont=dict(color="red", size=25),
        name="ROKIL",
        showlegend=False,
    )
)

# Update layout for the map
fig.update_layout(
    width=1000,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=8,
        center=dict(
            lat=navaids["ROKIL"].latitude,
            lon=navaids["ROKIL"].longitude,
        ),
    ),
    legend=dict(
        x=0.65,
        y=0.97,
        traceorder="normal",
        font=dict(size=25),
        bgcolor="rgba(255, 255, 255, 0.7)",
    ),
)

# Show the figure
fig.show()

#### Histogram

In [ ]:
t_ROKIL = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/rokil_ga_th_wp.parquet")

# Generate data for histogram --------------------------------------------------
# Empty lists
ids = []
distances = []
timestamp = []

for flight in tqdm(t_ROKIL):
    # determine distance form BELUS to RWY04 TH
    try:
        distance = (
            flight
            .cumulative_distance()
            .data.cumdist.iloc[-1]
        )
        # Append distance and is to lists
        distances.append(distance)
        ids.append(flight.flight_id)
        timestamp.append(flight.start)
    except:
        pass

# Turn lists into dataframe
df = pd.DataFrame(
    {"flight_id": ids, "distance": distances, "timestamp_utc": timestamp}
)

# Calculate the 99th percentile and the median
percentile_99 = np.percentile(df.distance, 99)
median = np.median(df.distance)

# Define the STAR distance
star_distance = 80.3

# Plot the histogram -----------------------------------------------------------
# Create the figure
fig = go.Figure()

# Add the histogram trace
fig.add_trace(
    go.Histogram(
        x=df.distance,
        xbins=dict(
            start=0,
            end=int(max(df.distance)),
            size=1,
        ),
        marker=dict(color="#1f77b4"),
        name="Observed distances",
        showlegend=True,
    ),
)

# Add a line indicating median
fig.add_trace(
    go.Scatter(
        x=[median, median],
        y=[-500, 3000],
        mode="lines",
        line=dict(color="darkblue", width=3, dash="dash"),
        name="Median of observed distances",
        showlegend=True,
    )
)

fig.add_annotation(
    x=median,
    y=-0.1,
    text=f"{median:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="darkblue"),
    xanchor="center",
)

# Add a line indicating the 99th percentile
fig.add_trace(
    go.Scatter(
        x=[percentile_99, percentile_99],
        y=[-500, 3000],
        mode="lines",
        line=dict(color="#ff7f0e", width=3, dash="dash"),
        name="99th Percentile of observed distances",
        showlegend=True,
    )
)

fig.add_annotation(
    x=percentile_99,
    y=-0.1,
    text=f"{percentile_99:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#ff7f0e"),
    xanchor="center",
)

# Add the line for "Full STAR distance"
fig.add_trace(
    go.Scatter(
        x=[star_distance, star_distance],
        y=[-100, 3000],
        mode="lines",
        line=dict(color="#d62728", width=3, dash="dash"),
        name="Full STAR distance",
        showlegend=True,
        legendgroup="Histogram",
    )
)

fig.add_annotation(
    x=star_distance,
    y=-0.1,
    text=f"{star_distance:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#d62728"),
    xanchor="center",
)

# Update the x-axis
fig.update_xaxes(
    range=[40, 100],
    title_text="Distance flown from ROKIL to THR RWY26L/R [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

# Update the y-axis
fig.update_yaxes(
    title_text="Count",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=30,
)

# Update layout to place legend outside the plot area
fig.update_layout(
    width=2000,
    height=800,
    margin=dict(l=50, r=300, t=40, b=100),
    legend=dict(
        font=dict(size=30),
        yanchor="top",
        xanchor="left",
        x=1.02,
        y=1,
        tracegroupgap=10
    ),
    xaxis=dict(
        showgrid=True,
    ),
    yaxis=dict(
        range=[-10, 100],
    ),
)

# Show the figure
fig.show()

#### Boxplot

In [ ]:
expected_star_distance = 72

# Add required columns
df["timestamp_lt"] = df["timestamp_utc"].dt.tz_convert("Europe/Berlin")
df["hour_utc"] = df.timestamp_utc.dt.hour
df["hour_lt"] = df.timestamp_lt.dt.hour

# Create a box plot using Plotly Express
fig = px.box(
    df,
    x="hour_lt",
    y="distance",
    points="outliers",
    labels={"hour_lt": "Hour of the day (local time)", "distance": "Distance [NM]"},
)

# Line full STAR distance
fig.add_trace(go.Scatter(
    x=[-0.5, df['hour_lt'].max() + 1],
    y=[star_distance, star_distance],
    mode="lines",
    line=dict(color="red", dash="dash", width=2),
    name="Full STAR distance",
    showlegend=True,
))

# Line expected STAR distance
fig.add_trace(go.Scatter(
    x=[-0.5, df['hour_lt'].max() + 1],
    y=[expected_star_distance, expected_star_distance],
    mode="lines",
    line=dict(color="green", dash="dash", width=2),
    name="Expected STAR distance as per AIP",
    showlegend=True,
))

# Update layout for better styling
fig.update_yaxes(
    title_text="Distance Flown from ROKIL to THR RWY26L/26R [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

fig.update_xaxes(
    title_text="Hour of the day (local time)",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

fig.update_layout(
    width=1000,
    height=1000,
    title=None,
    xaxis=dict(
        range=[3, 24],
        tickmode="linear",
        tick0=0,
        dtick=1,
        showgrid=True,
    ),
    yaxis=dict(
        showgrid=True,
        range=[0, 100],
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=True,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor="right",
        yanchor="top",
        bgcolor="rgba(255, 255, 255, 0.7)",
        font=dict(size=25),
    )
)

# Show the figure
fig.show()


***
### BETOS1B
***

#### Reduction to flights passing BETOS

In [ ]:
t_BETOS1B = (
    t_26.iterate_lazy()
    .pipe(hf.aligned_navpoint, "BETOS")
    .eval(desc="aligned_BETOS", max_workers=20)
)

t_BETOS1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/betos.parquet"
)

#### Remove go-arounds

In [ ]:
t_BETOS1B = (
    t_BETOS1B.iterate_lazy()
    .pipe(hf.remove_ga, "LSGG")
    .eval(desc="removing GAs", max_workers=20)
)

t_BETOS1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/betos_ga.parquet"
)

#### Crop at threshold

In [ ]:
t_BETOS1B_26L = t_BETOS1B.query("rwy == '26L'")
t_BETOS1B_26R = t_BETOS1B.query("rwy == '26R'")

t_BETOS1B_26L = (
    t_BETOS1B_26L.iterate_lazy()
    .pipe(hf.crop_after_th, "EDDM", "26L")
    .eval(desc="Cropping at TH 26L", max_workers=20)
)

t_BETOS1B_26R = (
    t_BETOS1B_26R.iterate_lazy()
    .pipe(hf.crop_after_th, "EDDM", "26R")
    .eval(desc="Cropping at TH 26R", max_workers=20)
)

t_BETOS1B = t_BETOS1B_26L + t_BETOS1B_26R

t_BETOS1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/betos_ga_th.parquet"
)

#### Crop before waypoint

In [ ]:
t_BETOS1B = (
    t_BETOS1B.iterate_lazy()
    .pipe(hf.crop_before_wp, "BETOS")
    .eval(desc="cropped_before_BETOS", max_workers=20)
)

t_BETOS1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/betos_ga_th_wp.parquet"
)

#### Map-plot

In [ ]:
# Load the data
t_BETOS = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/betos_ga_th_wp.parquet")

# Create a single figure for the map
fig = go.Figure()

# Add BETOS STAR 26L
betos_lat = [
    navaids["BETOS"].latitude,
    navaids["DM452"].latitude,
    navaids["DM459"].latitude,
    navaids["DM449"].latitude,
    airports["EDDM"].runways.data.query(f"name == '26L'").latitude.iloc[0],
]
betos_lon = [
    navaids["BETOS"].longitude,
    navaids["DM452"].longitude,
    navaids["DM459"].longitude,
    navaids["DM449"].longitude,
    airports["EDDM"].runways.data.query(f"name == '26L'").longitude.iloc[0],
]
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=betos_lat,
        lon=betos_lon,
        name="BETOS1B STAR",
        line=dict(width=10, color="red"),
        opacity=0.4,
        showlegend=True,
    )
)

# Add ROKIL STAR 26R
betos_lat = [
    navaids["DM449"].latitude,
    navaids["DM439"].latitude,
    airports["EDDM"].runways.data.query(f"name == '26R'").latitude.iloc[0],
]
betos_lon = [
    navaids["DM449"].longitude,
    navaids["DM439"].longitude,
    airports["EDDM"].runways.data.query(f"name == '26R'").longitude.iloc[0],
]
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=betos_lat,
        lon=betos_lon,
        name="BETOS STAR",
        line=dict(width=10, color="red"),
        opacity=0.4,
        showlegend=False,
    )
)

# Add observed trajectories
i = 0
for flight in t_BETOS.sample(10):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=1.5, color="#757ef3"),
            opacity=0.5,
            name="Observed trajectories",
            showlegend=False if i > 0 else True,
        )
    )
    i += 1

# Add BETOS navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["BETOS"].latitude],
        lon=[navaids["BETOS"].longitude],
        marker=dict(size=10, color="red"),
        text=["BETOS"],
        textposition="top left",
        textfont=dict(color="red", size=25),
        name="BETOS",
        showlegend=False,
    )
)

# Update layout for the map
fig.update_layout(
    width=1000,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=8,
        center=dict(
            lat=navaids["ROKIL"].latitude,
            lon=navaids["ROKIL"].longitude,
        ),
    ),
    legend=dict(
        x=0.65,
        y=0.97,
        traceorder="normal",
        font=dict(size=25),
        bgcolor="rgba(255, 255, 255, 0.7)",
    ),
)

# Show the figure
fig.show()

#### Histogram

In [ ]:
# Load the data
t_BETOS = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/betos_ga_th_wp.parquet")

# Generate data for histogram --------------------------------------------------
# Empty lists
ids = []
distances = []
timestamp = []

for flight in tqdm(t_BETOS):
    # determine distance form BELUS to RWY04 TH
    distance = (
        flight
        .cumulative_distance()
        .data.cumdist.iloc[-1]
    )
    # Append distance and is to lists
    distances.append(distance)
    ids.append(flight.flight_id)
    timestamp.append(flight.start)

# Turn lists into dataframe
df = pd.DataFrame(
    {"flight_id": ids, "distance": distances, "timestamp_utc": timestamp}
)

# Calculate the 99th percentile and the median
percentile_99 = np.percentile(df.distance, 99)
median = np.median(df.distance)

# Define the STAR distance
star_distance = 77

# Plot the histogram -----------------------------------------------------------
# Create the figure
fig = go.Figure()

# Add the histogram trace
fig.add_trace(
    go.Histogram(
        x=df.distance,
        xbins=dict(
            start=0,
            end=int(max(df.distance)),
            size=1,
        ),
        marker=dict(color="#1f77b4"),
        name="Observed distances",
        showlegend=True,
    ),
)

# Add a line indicating median
fig.add_trace(
    go.Scatter(
        x=[median, median],
        y=[-500, 3000],
        mode="lines",
        line=dict(color="darkblue", width=3, dash="dash"),
        name="Median of observed distances",
        showlegend=True,
    )
)

fig.add_annotation(
    x=median,
    y=-0.1,
    text=f"{median:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="darkblue"),
    xanchor="center",
)

# Add a line indicating the 99th percentile
fig.add_trace(
    go.Scatter(
        x=[percentile_99, percentile_99],
        y=[-500, 3000],
        mode="lines",
        line=dict(color="#ff7f0e", width=3, dash="dash"),
        name="99th Percentile of observed distances",
        showlegend=True,
    )
)

fig.add_annotation(
    x=percentile_99,
    y=-0.1,
    text=f"{percentile_99:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#ff7f0e"),
    xanchor="center",
)

# Add the line for "Full STAR distance"
fig.add_trace(
    go.Scatter(
        x=[star_distance, star_distance],
        y=[-100, 3000],
        mode="lines",
        line=dict(color="#d62728", width=3, dash="dash"),
        name="Full STAR distance",
        showlegend=True,
        legendgroup="Histogram",
    )
)

fig.add_annotation(
    x=star_distance,
    y=-0.1,
    text=f"{star_distance:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#d62728"),
    xanchor="center",
)

# Update the x-axis
fig.update_xaxes(
    range=[40, 100],
    title_text="Distance flown from BETOS to THR RWY26L/R [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

# Update the y-axis
fig.update_yaxes(
    title_text="Count",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=30,
)

# Update layout to place legend outside the plot area
fig.update_layout(
    width=2000,
    height=800,
    margin=dict(l=50, r=300, t=40, b=100),
    legend=dict(
        font=dict(size=30),
        yanchor="top",
        xanchor="left",
        x=1.02,
        y=1,
        tracegroupgap=10
    ),
    xaxis=dict(
        showgrid=True,
    ),
    yaxis=dict(
        range=[-10, 100],
    ),
)

# Show the figure
fig.show()

#### Boxplot

In [ ]:
expected_star_distance = 68

# Add required columns
df["timestamp_lt"] = df["timestamp_utc"].dt.tz_convert("Europe/Berlin")
df["hour_utc"] = df.timestamp_utc.dt.hour
df["hour_lt"] = df.timestamp_lt.dt.hour

# Create a box plot using Plotly Express
fig = px.box(
    df,
    x="hour_lt",
    y="distance",
    points="outliers",
    labels={"hour_lt": "Hour of the day (local time)", "distance": "Distance [NM]"},
)

# Line full STAR distance
fig.add_trace(go.Scatter(
    x=[-0.5, df['hour_lt'].max() + 1],
    y=[star_distance, star_distance],
    mode="lines",
    line=dict(color="red", dash="dash", width=2),
    name="Full STAR distance",
    showlegend=True,
))

# Line expected STAR distance
fig.add_trace(go.Scatter(
    x=[-0.5, df['hour_lt'].max() + 1],
    y=[expected_star_distance, expected_star_distance],
    mode="lines",
    line=dict(color="green", dash="dash", width=2),
    name="Expected STAR distance as per AIP",
    showlegend=True,
))

# Update layout for better styling
fig.update_yaxes(
    title_text="Distance Flown from BETOS to THR RWY26L/26R [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

fig.update_xaxes(
    title_text="Hour of the day (local time)",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

fig.update_layout(
    width=1000,
    height=1000,
    title=None,
    xaxis=dict(
        range=[3, 24],
        tickmode="linear",
        tick0=0,
        dtick=1,
        showgrid=True,
    ),
    yaxis=dict(
        showgrid=True,
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=True,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor="right",
        yanchor="top",
        bgcolor="rgba(255, 255, 255, 0.7)",
        font=dict(size=25),
    )
)

# Show the figure
fig.show()


***
### LANDU1B
***

#### Reduction to flights passing LANDU

In [ ]:
t_LANDU1B = (
    t_26.iterate_lazy()
    .pipe(hf.aligned_navpoint, "LANDU")
    .eval(desc="aligned_LANDU", max_workers=20)
)

t_LANDU1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/landu.parquet"
)

#### Remove go-arounds

In [ ]:
t_LANDU1B = (
    t_LANDU1B.iterate_lazy()
    .pipe(hf.remove_ga, "LSGG")
    .eval(desc="removing GAs", max_workers=20)
)

t_LANDU1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/landu_ga.parquet"
)

#### Crop at threshold

In [ ]:
t_LANDU1B_26L = t_LANDU1B.query("rwy == '26L'")
t_LANDU1B_26R = t_LANDU1B.query("rwy == '26R'")

t_LANDU1B_26L = (
    t_LANDU1B_26L.iterate_lazy()
    .pipe(hf.crop_after_th, "EDDM", "26L")
    .eval(desc="Cropping at TH 26L", max_workers=20)
)

t_LANDU1B_26R = (
    t_LANDU1B_26R.iterate_lazy()
    .pipe(hf.crop_after_th, "EDDM", "26R")
    .eval(desc="Cropping at TH 26R", max_workers=20)
)

t_LANDU1B = t_LANDU1B_26L + t_LANDU1B_26R

t_LANDU1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/landu_ga_th.parquet"
)

#### Crop before waypoint

In [ ]:
t_LANDU1B = (
    t_LANDU1B.iterate_lazy()
    .pipe(hf.crop_before_wp, "LANDU")
    .eval(desc="cropped_before_LANDU", max_workers=20)
)

t_LANDU1B.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/landu_ga_th_wp.parquet"
)

#### Map plot

In [ ]:
# Load the data
t_LANDU = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/landu_ga_th_wp.parquet")

# Create a single figure for the map
fig = go.Figure()

# Add LANDU STAR 26L
betos_lat = [
    navaids["BETOS"].latitude,
    navaids["DM452"].latitude,
    navaids["DM459"].latitude,
    navaids["DM449"].latitude,
    airports["EDDM"].runways.data.query(f"name == '26L'").latitude.iloc[0],
]
betos_lon = [
    navaids["BETOS"].longitude,
    navaids["DM452"].longitude,
    navaids["DM459"].longitude,
    navaids["DM449"].longitude,
    airports["EDDM"].runways.data.query(f"name == '26L'").longitude.iloc[0],
]
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=betos_lat,
        lon=betos_lon,
        name="LANDU1B STAR",
        line=dict(width=10, color="red"),
        opacity=0.4,
        showlegend=True,
    )
)

# Add LANDU STAR 26R
betos_lat = [
    navaids["DM449"].latitude,
    navaids["DM439"].latitude,
    airports["EDDM"].runways.data.query(f"name == '26R'").latitude.iloc[0],
]
betos_lon = [
    navaids["DM449"].longitude,
    navaids["DM439"].longitude,
    airports["EDDM"].runways.data.query(f"name == '26R'").longitude.iloc[0],
]
fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=betos_lat,
        lon=betos_lon,
        name="BETOS STAR",
        line=dict(width=10, color="red"),
        opacity=0.4,
        showlegend=False,
    )
)

# Add observed trajectories
i = 0
for flight in t_BETOS.sample(10):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=1.5, color="#757ef3"),
            opacity=0.5,
            name="Observed trajectories",
            showlegend=False if i > 0 else True,
        )
    )
    i += 1

# Add BETOS navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["BETOS"].latitude],
        lon=[navaids["BETOS"].longitude],
        marker=dict(size=10, color="red"),
        text=["BETOS"],
        textposition="top left",
        textfont=dict(color="red", size=25),
        name="BETOS",
        showlegend=False,
    )
)

# Update layout for the map
fig.update_layout(
    width=1000,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=8,
        center=dict(
            lat=navaids["ROKIL"].latitude,
            lon=navaids["ROKIL"].longitude,
        ),
    ),
    legend=dict(
        x=0.65,
        y=0.97,
        traceorder="normal",
        font=dict(size=25),
        bgcolor="rgba(255, 255, 255, 0.7)",
    ),
)

# Show the figure
fig.show()

#### Histogram

In [ ]:
# Load the data
t_LANDU = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/landu_ga_th_wp.parquet")

# Generate data for histogram --------------------------------------------------
# Empty lists
ids = []
distances = []
timestamp = []

for flight in tqdm(t_LANDU):
    # determine distance form BELUS to RWY04 TH
    distance = (
        flight
        .cumulative_distance()
        .data.cumdist.iloc[-1]
    )
    # Append distance and is to lists
    distances.append(distance)
    ids.append(flight.flight_id)
    timestamp.append(flight.start)

# Turn lists into dataframe
df = pd.DataFrame(
    {"flight_id": ids, "distance": distances, "timestamp_utc": timestamp}
)

# Calculate the 99th percentile and the median
percentile_99 = np.percentile(df.distance, 99)
median = np.median(df.distance)

# Define the STAR distance
star_distance = 77

# Plot the histogram -----------------------------------------------------------
# Create the figure
fig = go.Figure()

# Add the histogram trace
fig.add_trace(
    go.Histogram(
        x=df.distance,
        xbins=dict(
            start=0,
            end=int(max(df.distance)),
            size=1,
        ),
        marker=dict(color="#1f77b4"),
        name="Observed distances",
        showlegend=True,
    ),
)

# Add a line indicating median
fig.add_trace(
    go.Scatter(
        x=[median, median],
        y=[-500, 3000],
        mode="lines",
        line=dict(color="darkblue", width=3, dash="dash"),
        name="Median of observed distances",
        showlegend=True,
    )
)

fig.add_annotation(
    x=median,
    y=-0.1,
    text=f"{median:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="darkblue"),
    xanchor="center",
)

# Add a line indicating the 99th percentile
fig.add_trace(
    go.Scatter(
        x=[percentile_99, percentile_99],
        y=[-500, 3000],
        mode="lines",
        line=dict(color="#ff7f0e", width=3, dash="dash"),
        name="99th Percentile of observed distances",
        showlegend=True,
    )
)

fig.add_annotation(
    x=percentile_99,
    y=-0.1,
    text=f"{percentile_99:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#ff7f0e"),
    xanchor="center",
)

# Add the line for "Full STAR distance"
fig.add_trace(
    go.Scatter(
        x=[star_distance, star_distance],
        y=[-100, 3000],
        mode="lines",
        line=dict(color="#d62728", width=3, dash="dash"),
        name="Full STAR distance",
        showlegend=True,
        legendgroup="Histogram",
    )
)

fig.add_annotation(
    x=star_distance,
    y=-0.1,
    text=f"{star_distance:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#d62728"),
    xanchor="center",
)

# Update the x-axis
fig.update_xaxes(
    range=[40, 100],
    title_text="Distance flown from LANDU to THR RWY26L/R [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

# Update the y-axis
fig.update_yaxes(
    title_text="Count",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=30,
)

# Update layout to place legend outside the plot area
fig.update_layout(
    width=2000,
    height=800,
    margin=dict(l=50, r=300, t=40, b=100),
    legend=dict(
        font=dict(size=30),
        yanchor="top",
        xanchor="left",
        x=1.02,
        y=1,
        tracegroupgap=10
    ),
    xaxis=dict(
        showgrid=True,
    ),
    yaxis=dict(
        range=[-10, 100],
    ),
)

# Show the figure
fig.show()

***
### NAPSA 4A

#### Reduce to flights passing NAPSA

In [ ]:
t_NAPSA4 = (
    t_26.iterate_lazy()
    .pipe(hf.aligned_navpoint, "NAPSA")
    .eval(desc="aligned_NAPSA", max_workers=20)
)
t_NAPSA4.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/napsa.parquet"
)

#### Remove go-arounds

In [ ]:
t_NAPSA4 = (
    t_NAPSA4.iterate_lazy()
    .pipe(hf.remove_ga, "EDDM")
    .eval(desc="removing GAs", max_workers=20)
)
t_NAPSA4.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/napsa_ga.parquet"
)

#### Crop at threshold

In [ ]:
t_NAPSA4_26L = t_NAPSA4.query("rwy == '26L'")
t_NAPSA4_26R = t_NAPSA4.query("rwy == '26R'")

t_NAPSA4_26L = (
    t_NAPSA4_26L.iterate_lazy()
    .pipe(hf.crop_after_th, "EDDM", "26L")
    .eval(desc="Cropping at TH 26L", max_workers=20)
)

t_NAPSA4_26R = (
    t_NAPSA4_26R.iterate_lazy()
    .pipe(hf.crop_after_th, "EDDM", "26R")
    .eval(desc="Cropping at TH 26R", max_workers=20)
)

t_NAPSA4 = t_NAPSA4_26L + t_NAPSA4_26R

t_NAPSA4.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/napsa_ga_th.parquet"
)

#### Crop before waypoint

In [ ]:
t_NAPSA4 = (
    t_NAPSA4.iterate_lazy()
    .pipe(hf.crop_before_wp, "NAPSA")
    .eval(desc="cropped_before_NAPSA", max_workers=20)
)

t_NAPSA4.to_parquet(
    "/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/napsa_ga_th_wp.parquet"
)

#### Map-plot

In [ ]:
# Create a single figure for the map
fig = go.Figure()

# Add NAPSA1B STAR 26L
napsa_lat = [
    navaids["NAPSA"].latitude,
    navaids["DM463"].latitude,
    navaids["DM452"].latitude,
    navaids["DM459"].latitude,
    navaids["DM449"].latitude,
    airports["EDDM"].runways.data.query(f"name == '26L'").latitude.iloc[0],
]
napsa_lon = [
    navaids["NAPSA"].longitude,
    navaids["DM463"].longitude,
    navaids["DM452"].longitude,
    navaids["DM459"].longitude,
    navaids["DM449"].longitude,
    airports["EDDM"].runways.data.query(f"name == '26L'").longitude.iloc[0],
]

fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=napsa_lat,
        lon=napsa_lon,
        name="NAPSA STAR RWY 26L/R",
        line=dict(width=10, color="red"),
        opacity=0.4,
        showlegend=True,
    )
)

# Add NAPSA1B STAR 26R
napsa_lat = [
    navaids["DM449"].latitude,
    navaids["DM439"].latitude,
    airports["EDDM"].runways.data.query(f"name == '26R'").latitude.iloc[0],
]
napsa_lon = [
    navaids["DM449"].longitude,
    navaids["DM439"].longitude,
    airports["EDDM"].runways.data.query(f"name == '26R'").longitude.iloc[0],
]

fig.add_trace(
    go.Scattermapbox(
        mode="lines",
        lat=napsa_lat,
        lon=napsa_lon,
        name="DEGES SID",
        line=dict(width=10, color="red"),
        opacity=0.4,
        showlegend=False,
    )
)

# Add flight trajectories
i = 0
for flight in t_NAPSA4.sample(50):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=2, color="#757ef3"),
            opacity=0.5,
            name="Observed trajectories",
            showlegend=False if i > 0 else True,
        )
    )
    i += 1

# Add NAPSA navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["NAPSA"].latitude],
        lon=[navaids["NAPSA"].longitude],
        marker=dict(size=12, color="red"),
        text=["NAPSA"],
        textposition="bottom right",
        textfont=dict(color="red", size=25),
        name="NAPSA",
        showlegend=False,
    )
)

# Update layout for the map and legend
fig.update_layout(
    width=1000,
    height=1000,
    margin=dict(l=0, r=0, t=0, b=0),
    mapbox=dict(
        style="carto-positron",
        zoom=8,
        center=dict(
            lat=navaids["NAPSA"].latitude,
            lon=navaids["NAPSA"].longitude,
        ),
    ),
    # Configure the legend to be inside the map plot
    legend=dict(
        x=0.65,
        y=0.97,
        traceorder="normal",
        font=dict(size=25),
        bgcolor="rgba(255, 255, 255, 0.7)",  # Add some transparency for better visibility
    ),
)

# Show the figure
fig.show()


In [ ]:
t_NAPSA = Traffic.from_file(f"/mnt/beegfs/store/Projects_CRM/STAR_paper/EDDM/processed/napsa_ga_th_wp.parquet")

# Generate data for histogram --------------------------------------------------
# Empty lists
ids = []
distances = []
timestamp = []

for flight in tqdm(t_NAPSA):
    distance = (
        flight
        .cumulative_distance()
        .data.cumdist.iloc[-1]
    )
    # Append distance and is to lists
    distances.append(distance)
    ids.append(flight.flight_id)
    timestamp.append(flight.start)

# Turn lists into dataframe
df = pd.DataFrame(
    {"flight_id": ids, "distance": distances, "timestamp_utc": timestamp}
)

# Calculate the 99th percentile and the median
percentile_99 = np.percentile(df.distance, 99)
median = np.median(df.distance)

# Define the STAR distance
star_distance = 108.8

# Plot the histogram -----------------------------------------------------------
# Create the figure
fig = go.Figure()

# Add the histogram trace
fig.add_trace(
    go.Histogram(
        x=df.distance,
        xbins=dict(
            start=0,
            end=int(max(df.distance)),
            size=1,
        ),
        marker=dict(color="#1f77b4"),
        name="Observed distances",
        showlegend=True,
    ),
)

# Add a line indicating median
fig.add_trace(
    go.Scatter(
        x=[median, median],
        y=[-500, 3000],
        mode="lines",
        line=dict(color="darkblue", width=3, dash="dash"),
        name="Median of observed distances",
        showlegend=True,
    )
)

fig.add_annotation(
    x=median,
    y=-0.1,
    text=f"{median:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="darkblue"),
    xanchor="center",
)

# Add a line indicating the 99th percentile
fig.add_trace(
    go.Scatter(
        x=[percentile_99, percentile_99],
        y=[-500, 3000],
        mode="lines",
        line=dict(color="#ff7f0e", width=3, dash="dash"),
        name="99th Percentile of observed distances",
        showlegend=True,
    )
)

fig.add_annotation(
    x=percentile_99,
    y=-0.1,
    text=f"{percentile_99:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#ff7f0e"),
    xanchor="center",
)

# Add the line for "Full STAR distance"
fig.add_trace(
    go.Scatter(
        x=[star_distance, star_distance],
        y=[-100, 3000],
        mode="lines",
        line=dict(color="#d62728", width=3, dash="dash"),
        name="Full STAR distance",
        showlegend=True,
        legendgroup="Histogram",
    )
)

fig.add_annotation(
    x=star_distance,
    y=-0.1,
    text=f"{star_distance:.1f}",
    showarrow=False,
    yref="paper",
    font=dict(size=25, color="#d62728"),
    xanchor="center",
)

# Update the x-axis
fig.update_xaxes(
    range=[0, 150],
    title_text="Distance flown from ROKIL to THR RWY26L/R [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

# Update the y-axis
fig.update_yaxes(
    title_text="Count",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=30,
)

# Update layout to place legend outside the plot area
fig.update_layout(
    width=2000,
    height=800,
    margin=dict(l=50, r=300, t=40, b=100),
    legend=dict(
        font=dict(size=30),
        yanchor="top",
        xanchor="left",
        x=1.02,
        y=1,
        tracegroupgap=10
    ),
    xaxis=dict(
        showgrid=True,
    ),
    yaxis=dict(
        range=[-10, 450],
    ),
)

# Show the figure
fig.show()

In [ ]:
# Generate required columns
df["timestamp_lt"] = df["timestamp_utc"].dt.tz_convert("Europe/Berlin")
df["hour_utc"] = df.timestamp_utc.dt.hour
df["hour_lt"] = df.timestamp_lt.dt.hour
df

# Create a box plot using Plotly Express
fig = px.box(
    df,
    x="hour_lt",  # Hour of the day on the x-axis
    y="distance",  # Distance flown on the y-axis
    points="outliers",  # Show outliers
    title="Distance Flown from NAPSA to THR RWY26L/26R [NM]",
    labels={"hour_lt": "Hour of the day (local time)", "distance": "Distance [NM]"},
)

fig.add_trace(go.Scatter(
    x=[-0.5, df['hour_lt'].max() + 1],
    y=[108.8, 108.8],
    mode="lines",
    line=dict(color="red", dash="dash", width=2),
    name="Full STAR distance",
    showlegend=True,
))

fig.add_trace(go.Scatter(
    x=[-0.5, df['hour_lt'].max() + 1],
    y=[47, 47],
    mode="lines",
    line=dict(color="green", dash="dash", width=2),
    name="Expected STAR distance as per AIP",
    showlegend=True,
))

fig.update_yaxes(
    title_text="Distance Flown from NAPSA to THR RWY26L/26R [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

fig.update_xaxes(
    title_text="Hour of the day (local time)",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=40,
)

# Update layout for better styling
fig.update_layout(
    width=1000,
    height=1000,
    title=None,
    xaxis=dict(
        range=[3, 24],
        tickmode="linear",
        tick0=0,
        dtick=1,
        showgrid=True,
    ),
    yaxis=dict(
        showgrid=True,
        range=[0, 130],
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    showlegend=True,
    legend=dict(
        x=0.95,
        y=0.95,
        xanchor="right",
        yanchor="top",
        bgcolor="rgba(255, 255, 255, 0.7)",
        font=dict(size=25),
    )
)

# Show the figure
fig.show()
